In [83]:
import json
import glob
import itertools
from collections import defaultdict

from pathlib import Path
import numpy as np
import pandas as pd

from qiskit import transpile, QuantumCircuit
from qiskit.providers.fake_provider import FakeLima
from qiskit.primitives import Estimator, BackendEstimator
from qiskit.circuit.random import random_circuit
from qiskit.quantum_info import SparsePauliOp

import torch
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.nn.functional import dropout

from torch_geometric.nn import (
    GCNConv,
    TransformerConv,
    GATv2Conv,
    global_mean_pool,
    Linear,
    ChebConv,
    SAGEConv,
    ASAPooling,
    dense_diff_pool,
    avg_pool_neighbor_x
)
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.utils import to_dense_adj, to_dense_batch

from tqdm.notebook import tqdm_notebook
import matplotlib.pyplot as plt
import seaborn as sns

from blackwater.data.loaders.exp_val import CircuitGraphExpValMitigationDataset
from blackwater.data.generators.exp_val import exp_value_generator
from blackwater.data.utils import generate_random_pauli_sum_op
from blackwater.library.ngem.estimator import ngem
from blackwater.data.utils import (
    generate_random_pauli_sum_op,
    create_estimator_meas_data,
    circuit_to_graph_data_json,
    get_backend_properties_v1,
    encode_pauli_sum_op,
    create_meas_data_from_estimators
)
from blackwater.data.generators.exp_val import ExpValueEntry
from blackwater.metrics.improvement_factor import improvement_factor, Trial, Problem

from qiskit_aer.primitives import Estimator as AerEstimator
from qiskit_aer.noise import NoiseModel
import qiskit_aer.noise as noise

from qiskit.providers.models import BackendProperties

from pprint import pprint
import copy

In [15]:
backend = FakeLima()
properties = get_backend_properties_v1(backend)

In [80]:
files = sorted(glob.glob('./data/circ_linked_to_pyg_data/*.txt'))
files

['./data/circ_linked_to_pyg_data/qasm_11_-0.1.txt',
 './data/circ_linked_to_pyg_data/qasm_11_-0.2.txt',
 './data/circ_linked_to_pyg_data/qasm_11_-0.3.txt',
 './data/circ_linked_to_pyg_data/qasm_11_-0.4.txt',
 './data/circ_linked_to_pyg_data/qasm_11_-0.5.txt',
 './data/circ_linked_to_pyg_data/qasm_11_-0.6.txt',
 './data/circ_linked_to_pyg_data/qasm_11_-0.7.txt',
 './data/circ_linked_to_pyg_data/qasm_11_-0.8.txt',
 './data/circ_linked_to_pyg_data/qasm_11_-0.9.txt',
 './data/circ_linked_to_pyg_data/qasm_11_-1.0.txt',
 './data/circ_linked_to_pyg_data/qasm_11_0.0.txt',
 './data/circ_linked_to_pyg_data/qasm_11_0.1.txt',
 './data/circ_linked_to_pyg_data/qasm_11_0.2.txt',
 './data/circ_linked_to_pyg_data/qasm_11_0.3.txt',
 './data/circ_linked_to_pyg_data/qasm_11_0.4.txt',
 './data/circ_linked_to_pyg_data/qasm_11_0.5.txt',
 './data/circ_linked_to_pyg_data/qasm_11_0.6.txt',
 './data/circ_linked_to_pyg_data/qasm_11_0.7.txt',
 './data/circ_linked_to_pyg_data/qasm_11_0.8.txt',
 './data/circ_linked_

In [ ]:
circs = defaultdict(list)

for file in tqdm_notebook(files, leave=False):
    print(file)
    depth = float(file.split('_')[-2])
    qc_list = []
    with open(file, "r") as f:
        lines = f.read().split('-----')
        print(len(lines) - 1)
        for i, line in enumerate(lines):
            splitted = line.split('\n')
            valid = splitted[3:]
            if len(valid) and valid[0].startswith('qasm'):
                valid = valid[1:]
            elif not len(valid):
                continue
            try:
                exp_val = float(splitted[1])
            except:
                exp_val = float(splitted[2])
            qasm_str = '\n'.join(valid)
            qc = QuantumCircuit.from_qasm_str(qasm_str)
            qc_list.append((exp_val, qc))
    print(len(qc_list))
    circs[depth] += qc_list

  0%|          | 0/300 [00:00<?, ?it/s]

./data/circ_linked_to_pyg_data/qasm_11_-0.1.txt
100
100
./data/circ_linked_to_pyg_data/qasm_11_-0.2.txt
100
100
./data/circ_linked_to_pyg_data/qasm_11_-0.3.txt
92
92
./data/circ_linked_to_pyg_data/qasm_11_-0.4.txt
94
94
./data/circ_linked_to_pyg_data/qasm_11_-0.5.txt
91
91
./data/circ_linked_to_pyg_data/qasm_11_-0.6.txt
100
100
./data/circ_linked_to_pyg_data/qasm_11_-0.7.txt
100
100
./data/circ_linked_to_pyg_data/qasm_11_-0.8.txt
100
100
./data/circ_linked_to_pyg_data/qasm_11_-0.9.txt
100
100
./data/circ_linked_to_pyg_data/qasm_11_-1.0.txt
100
100
./data/circ_linked_to_pyg_data/qasm_11_0.0.txt
100
100
./data/circ_linked_to_pyg_data/qasm_11_0.1.txt
100
100
./data/circ_linked_to_pyg_data/qasm_11_0.2.txt
100
100
./data/circ_linked_to_pyg_data/qasm_11_0.3.txt
94
94
./data/circ_linked_to_pyg_data/qasm_11_0.4.txt
96
96
./data/circ_linked_to_pyg_data/qasm_11_0.5.txt
90
90
./data/circ_linked_to_pyg_data/qasm_11_0.6.txt
96
96
./data/circ_linked_to_pyg_data/qasm_11_0.7.txt
100
100
./data/circ_li

In [ ]:
with open('./data/circ_parsed_pyg_data/parsed.json') as out:
    json.dump(circs, out)